In [ ]:
# run this cell, then restart the runtime before continuing
!pip install -q transformers
!pip install -q datasets

     |████████████████████████████████| 2.1MB 7.5MB/s 
     |████████████████████████████████| 890kB 36.7MB/s 
     |████████████████████████████████| 3.2MB 51.9MB/s 
     |████████████████████████████████| 194kB 6.9MB/s 
     |████████████████████████████████| 245kB 8.7MB/s 
     |████████████████████████████████| 112kB 14.8MB/s 


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification,Trainer, TrainingArguments
from transformers import TFBertForSequenceClassification, TFTrainer, TFTrainingArguments
import torch
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import tqdm
from datasets import load_dataset
import tensorflow as tf

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')
model = AutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased',num_labels =4)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
tokenizer.save_pretrained("drive/MyDrive/cs329s/bert_tokenizer")

('drive/MyDrive/cs329s/bert_tokenizer/tokenizer_config.json',
 'drive/MyDrive/cs329s/bert_tokenizer/special_tokens_map.json',
 'drive/MyDrive/cs329s/bert_tokenizer/vocab.txt',
 'drive/MyDrive/cs329s/bert_tokenizer/added_tokens.json')

In [ ]:
# load dataset
pubhealth = load_dataset('health_fact')

Using custom data configuration default


Dataset health_fact downloaded and prepared to /root/.cache/huggingface/datasets/health_fact/default/1.1.0/f6edecc77e6780a21be2f0a8882c78808b1c1a4c1f50e717a5acd1d5a34af76a. Subsequent calls will reuse this data.


In [ ]:
def tokenize(batch):
    return tokenizer(batch['main_text'], padding=True, truncation=True)

def clean_label(batch):
  labels = batch['label']
  cleaned = [3 if x==-1 else x for x in labels]
  return {'label':cleaned}

train_dataset, val_dataset,test_dataset = load_dataset('health_fact', split=['train', 'validation','test'])

train_dataset = train_dataset.map(tokenize, batched=True, batch_size=len(train_dataset))
val_dataset = val_dataset.map(tokenize, batched=True, batch_size=len(train_dataset))
test_dataset = test_dataset.map(tokenize, batched=True, batch_size=len(train_dataset))

train_dataset = train_dataset.map(clean_label, batched=True, batch_size=len(train_dataset))
val_dataset = val_dataset.map(clean_label, batched=True, batch_size=len(train_dataset))
test_dataset = test_dataset.map(clean_label, batched=True, batch_size=len(train_dataset))


# train_dataset.set_format(type='tensorflow', columns=['input_ids', 'attention_mask', 'label'])
# val_dataset.set_format(type='tensorflow', columns=['input_ids', 'attention_mask', 'label'])
# test_dataset.set_format(type='tensorflow', columns=['input_ids', 'attention_mask', 'label'])
# dataloader_train = torch.utils.data.DataLoader(train_dataset, batch_size=32)
# val_dataset.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'], device='cuda')
# dataloader_val = torch.utils.data.DataLoader(val_dataset, batch_size=32)

Using custom data configuration default


Dataset health_fact downloaded and prepared to /root/.cache/huggingface/datasets/health_fact/default/1.1.0/f6edecc77e6780a21be2f0a8882c78808b1c1a4c1f50e717a5acd1d5a34af76a. Subsequent calls will reuse this data.


In [ ]:
features = {x: train_dataset[x].to_tensor(default_value=0, shape=[None, tokenizer.model_max_length]) for x in ['input_ids', 'attention_mask']}
tfdataset_train = tf.data.Dataset.from_tensor_slices((features, train_dataset["label"])).batch(32)

features = {x: val_dataset[x].to_tensor(default_value=0, shape=[None, tokenizer.model_max_length]) for x in ['input_ids', 'attention_mask']}
tfdataset_val = tf.data.Dataset.from_tensor_slices((features, val_dataset["label"])).batch(32)

features = {x: test_dataset[x].to_tensor(default_value=0, shape=[None, tokenizer.model_max_length]) for x in ['input_ids', 'attention_mask']}
tfdataset_test = tf.data.Dataset.from_tensor_slices((features, test_dataset["label"])).batch(32)

In [ ]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds)
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=5,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model='accuracy',
)

trainer = Trainer(
    model=model_fine_tuned ,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

In [ ]:
trainer.train()

Step,Training Loss
500,0.887000
1000,0.680700
1500,0.553500
2000,0.411500
2500,0.268300
3000,0.161700


TrainOutput(global_step=3075, training_loss=0.48493346516678976, metrics={'train_runtime': 2617.3937, 'train_samples_per_second': 1.175, 'total_flos': 10111745739816960, 'epoch': 5.0})

In [ ]:
trainer.evaluate()

Trainer is attempting to log a value of "[0.67362924 0.4        0.82893617 0.28571429]" of type <class 'numpy.ndarray'> for key "eval/f1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[0.68253968 0.3647541  0.84548611 0.32432432]" of type <class 'numpy.ndarray'> for key "eval/precision" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[0.66494845 0.44278607 0.8130217  0.25531915]" of type <class 'numpy.ndarray'> for key "eval/recall" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


{'eval_accuracy': 0.6850202429149798,
 'eval_f1': array([0.67362924, 0.4       , 0.82893617, 0.28571429]),
 'eval_loss': 1.2882983684539795,
 'eval_mem_cpu_alloc_delta': 148377,
 'eval_mem_cpu_peaked_delta': 1306655,
 'eval_mem_gpu_alloc_delta': 0,
 'eval_mem_gpu_peaked_delta': 2315848192,
 'eval_precision': array([0.68253968, 0.3647541 , 0.84548611, 0.32432432]),
 'eval_recall': array([0.66494845, 0.44278607, 0.8130217 , 0.25531915]),
 'eval_runtime': 21.8948,
 'eval_samples_per_second': 56.406,
 'init_mem_cpu_alloc_delta': 49032,
 'init_mem_cpu_peaked_delta': 18306,
 'init_mem_gpu_alloc_delta': 0,
 'init_mem_gpu_peaked_delta': 0}

In [ ]:
trainer.save_model("drive/MyDrive/cs329s/bert_fine_tuned")

In [ ]:
from transformers import TFDistilBertForSequenceClassification
tf_model = TFDistilBertForSequenceClassification.from_pretrained("drive/MyDrive/cs329s/bert_fine_tuned", from_pt=True)
tf_model.save_pretrained("drive/MyDrive/cs329s/bert_fine_tuned_tf")

All PyTorch model weights were used when initializing TFDistilBertForSequenceClassification.

All the weights of TFDistilBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


In [ ]:
model_fine_tuned = AutoModelForSequenceClassification.from_pretrained("drive/MyDrive/cs329s/bert_fine_tuned")
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')
# pipeline = TextClassificationPipeline(model = model_fine_tuned, tokenizer = tokenizer, return_all_scores = True)

In [ ]:
from transformers import TextClassificationPipeline
pipeline = TextClassificationPipeline(model = model, tokenizer = tokenizer, return_all_scores = True)
example = 'COVID is fake. Asymptomatic transmission is not real.'
pipeline(example)

[[{'label': 'LABEL_0', 'score': 0.9966179132461548},
  {'label': 'LABEL_1', 'score': 0.0013299603015184402},
  {'label': 'LABEL_2', 'score': 0.00127703370526433},
  {'label': 'LABEL_3', 'score': 0.0007750133518129587}]]